In [3]:
import numpy as np
import pandas as pd

from typing import List, Tuple, Dict
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import os
from pathlib import Path
import warnings
import logging
import sys

warnings.filterwarnings("ignore")

PROJECT_ROOT = Path.cwd().parent
sys.path.append(str(PROJECT_ROOT))

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

DATA_DIR = PROJECT_ROOT / "data" / "processed"
EVENTS_PROPS_FILE = DATA_DIR / "events_with_properties.csv"



In [4]:
def load(path):
    logging.info("Loading from %s", path)
    df = pd.read_csv(path)
    if 'timestamp'in df.columns:
        df['timestamp'] = pd.to_datetime(df['timestamp'])
    if 'first_event'in df.columns:
        df['first_event'] = pd.to_datetime(df['first_event'])
    if 'last_event'in df.columns:
        df['last_event'] = pd.to_datetime(df['last_event'])
    return df
events = load(EVENTS_PROPS_FILE)



In [5]:
events

,timestamp,visitorid,event,itemid,transactionid,price,available,categoryid,mean_price
0,2015-05-03 03:00:04.384,693516,addtocart,297662,NaN,11654.000000,NaN,NaN,11654.000000
1,2015-05-03 03:00:11.289,829044,view,60987,NaN,204120.000000,NaN,NaN,204120.000000
2,2015-05-03 03:00:13.048,652699,view,252860,NaN,NaN,NaN,NaN,NaN
3,2015-05-03 03:00:24.154,1125936,view,33661,NaN,32713.333333,NaN,NaN,32713.333333
4,2015-05-03 03:00:26.228,693516,view,297662,NaN,11654.000000,NaN,NaN,11654.000000
...,...,...,...,...,...,...,...,...,...
2748339,2015-09-18 02:59:24.029,472345,view,301436,NaN,21120.000000,NaN,1244.0,21120.000000
2748340,2015-09-18 02:59:25.499,1207677,view,310922,NaN,23760.000000,NaN,NaN,23760.000000
2748341,2015-09-18 02:59:34.109,255126,view,47467,NaN,134520.000000,0.0,491.0,134520.000000
2748342,2015-09-18 02:59:41.778,622226,view,345308,NaN,20400.000000,1.0,NaN,19840.000000


STRATEGIC DASHBOARD


MANAGER VIEW

In [3]:
import pandas as pd

df = events.copy()

df['timestamp'] = pd.to_datetime(df['timestamp'])
df['day'] = df['timestamp'].dt.date
df['month'] = df['timestamp'].dt.to_period('M')


daily_events = (
    df
    .groupby(['visitorid', 'day'])
    .agg(
        number_of_activity=('event', 'count'),
        number_of_view=('event', lambda x: (x == 'view').sum()),
        number_of_addtocart=('event', lambda x: (x == 'addtocart').sum())
    )
)

daily_transactions = (
    df[df['event'] == 'transaction']
    .groupby(['visitorid', 'day'])
    .agg(
        number_of_purchase=('transactionid', 'nunique'),
        revenue=('price', 'sum')
    )
)

daily_df = (
    daily_events
    .join(daily_transactions, how='left')
    .fillna({'number_of_purchase': 0, 'revenue': 0})
    .reset_index()
)

daily_df['month'] = pd.to_datetime(daily_df['day']).dt.to_period('M')


first_tx_day = (
    df[df['event'] == 'transaction']
    .groupby('visitorid')['day']
    .min()
)

first_add_day = (
    df[df['event'] == 'addtocart']
    .groupby('visitorid')['day']
    .min()
)

first_view_day = (
    df[df['event'] == 'view']
    .groupby('visitorid')['day']
    .min()
)

daily_df['first_tx_day'] = daily_df['visitorid'].map(first_tx_day)
daily_df['first_add_day'] = daily_df['visitorid'].map(first_add_day)
daily_df['first_view_day'] = daily_df['visitorid'].map(first_view_day)


daily_df['new_user_transaction'] = (
    daily_df['first_tx_day'].isna()
    | (daily_df['day'] < daily_df['first_tx_day'])
).astype(int)


daily_df['new_user_addtocart'] = (
    (daily_df['first_add_day'].isna() | (daily_df['day'] < daily_df['first_add_day']))
    &
    (daily_df['first_tx_day'].isna() | (daily_df['day'] < daily_df['first_tx_day']))
).astype(int)

daily_df['new_user_view'] = (
    (daily_df['first_view_day'].isna() | (daily_df['day'] < daily_df['first_view_day']))
    &
    (daily_df['first_add_day'].isna() | (daily_df['day'] < daily_df['first_add_day']))
    &
    (daily_df['first_tx_day'].isna() | (daily_df['day'] < daily_df['first_tx_day']))
).astype(int)


daily_df.drop(
    columns=['first_tx_day', 'first_add_day', 'first_view_day'],
    inplace=True
)

daily_df = daily_df[
    [
        'visitorid',
        'day',
        'month',
        'number_of_view',
        'number_of_addtocart',
        'number_of_purchase',
        'number_of_activity',
        'new_user_transaction',
        'new_user_addtocart',
        'new_user_view',
        'revenue'
    ]
]


In [1]:
events

NameError: name 'events' is not defined

In [29]:
import pandas as pd



df = events.copy()

df['timestamp'] = pd.to_datetime(df['timestamp'])
df = df.sort_values(['visitorid', 'timestamp'])

df['day'] = df['timestamp'].dt.date
df['month'] = df['timestamp'].dt.to_period('M')




df['had_view_before'] = (
    (df['event'] == 'view')
    .groupby(df['visitorid'])
    .cumsum()
    .shift(fill_value=0)
)

df['had_add_before'] = (
    (df['event'] == 'addtocart')
    .groupby(df['visitorid'])
    .cumsum()
    .shift(fill_value=0)
)

df['had_tx_before'] = (
    (df['event'] == 'transaction')
    .groupby(df['visitorid'])
    .cumsum()
    .shift(fill_value=0)
)




daily_fact = (
    df
    .groupby(['visitorid', 'day', 'month'])
    .agg(
        number_of_view=('event', lambda x: (x == 'view').sum()),
        number_of_addtocart=('event', lambda x: (x == 'addtocart').sum()),
        number_of_purchase=('event', lambda x: (x == 'transaction').sum()),
        number_of_activity = ('event', 'count'),
        revenue=('price', 'sum'),  # <-- revenue added

        had_view_before=('had_view_before', 'max'),
        had_add_before=('had_add_before', 'max'),
        had_tx_before=('had_tx_before', 'max'),
    )
    .reset_index()
)




daily_fact['new_user_transaction_day'] = (
    daily_fact['had_tx_before'] == 0
).astype(int)

daily_fact['new_user_addtocart_day'] = (
    (daily_fact['had_add_before'] == 0) &
    (daily_fact['had_tx_before'] == 0)
).astype(int)

daily_fact['new_user_view_day'] = (
    (daily_fact['had_view_before'] == 0) &
    (daily_fact['had_add_before'] == 0) &
    (daily_fact['had_tx_before'] == 0)
).astype(int)



daily_fact = daily_fact.sort_values(['visitorid', 'day'])

daily_fact['is_first_row_in_month'] = (
    daily_fact
    .groupby(['visitorid', 'month'])
    .cumcount()
    == 0
)

daily_fact['new_user_view_month'] = (
    daily_fact['is_first_row_in_month']
).astype(int)

daily_fact['new_user_addtocart_month'] = (
    daily_fact['is_first_row_in_month'] &
    (
        (daily_fact['number_of_addtocart'] > 0) |
        (daily_fact['number_of_purchase'] > 0)
    )
).astype(int)

daily_fact['new_user_transaction_month'] = (
    daily_fact['is_first_row_in_month'] &
    (daily_fact['number_of_purchase'] > 0)
).astype(int)


daily_df = daily_fact[
    [
        'visitorid',
        'day',
        'month',

        'number_of_view',
        'number_of_addtocart',
        'number_of_purchase',
        'number_of_activity',
        'revenue',

        # DAY flags
        'new_user_view_day',
        'new_user_addtocart_day',
        'new_user_transaction_day',

        # MONTH flags
        'new_user_view_month',
        'new_user_addtocart_month',
        'new_user_transaction_month',
    ]
]


In [31]:
daily_df = daily_df.sort_values(['visitorid', 'day'])
daily_df['cum_revenue'] = daily_df.groupby('visitorid')['revenue'].cumsum()
daily_df['cum_purchases'] = daily_df.groupby('visitorid')['number_of_purchase'].cumsum()
daily_df['cum_addtocart'] = daily_df.groupby('visitorid')['number_of_addtocart'].cumsum()
daily_df['cum_activity'] = daily_df.groupby('visitorid')['number_of_activity'].cumsum()
daily_df['revenue_tier'] = (
    daily_df
    .groupby('day')['cum_revenue']
    .transform(
        lambda x: pd.qcut(
            x.rank(method='first'),
            q=[0, 0.5, 0.7, 1.0],
            labels=['Low', 'Mid', 'High']
        )
    )
)
def assign_business_label(row):
    if row['revenue_tier'] == 'High' and row['cum_purchases'] >= 2:
        return 'Loyal'
    if (
        row['cum_purchases'] >= 2
        or row['cum_addtocart'] >= 3
        or row['cum_activity'] >= 5
    ):
        return 'Potential Loyal'

    return 'At Risk'


daily_df['business_label'] = daily_df.apply(assign_business_label, axis=1)
daily_df = daily_df[
    [
        'visitorid',
        'day',
        'month',

        'number_of_view',
        'number_of_addtocart',
        'number_of_purchase',
        'number_of_activity',
        'revenue',

        # DAY flags
        'new_user_view_day',
        'new_user_addtocart_day',
        'new_user_transaction_day',

        # MONTH flags
        'new_user_view_month',
        'new_user_addtocart_month',
        'new_user_transaction_month',
        
        'cum_revenue',
        'cum_purchases',
        'cum_addtocart',
        'cum_activity',
        'business_label'
    ]
]

In [32]:
daily_df['business_label'].value_counts()

business_label
At Risk            1493890
Potential Loyal     148612
Loyal                 6958
Name: count, dtype: int64

In [33]:
daily_df['new_user_transaction_day'].value_counts()

new_user_transaction_day
1    1622435
0      27025
Name: count, dtype: int64

In [18]:
daily_df['number_of_purchase'].sum()

np.float64(17172.0)

In [34]:
OUTPUT_DIR = PROJECT_ROOT / "data" / "processed" / "dashboard.csv"
daily_df.to_csv(OUTPUT_DIR, index = False)

In [9]:
OUTPUT_DIR = PROJECT_ROOT / "data" / "processed" / "dashboard.csv"
daily_df = pd.read_csv(OUTPUT_DIR)

In [11]:
daily_df['number_of_addtocart'].value_counts()

number_of_addtocart
0      1607899
1        31853
2         5212
3         1722
4          906
5          477
6          323
7          195
8          174
9          129
10         104
11          86
13          47
12          45
14          39
16          33
15          32
17          26
18          21
20          19
19          16
21          15
23          13
24           8
26           7
22           7
25           6
31           6
27           5
28           5
33           4
35           3
30           3
32           2
36           2
103          1
71           1
84           1
48           1
44           1
37           1
70           1
39           1
40           1
57           1
34           1
61           1
92           1
42           1
43           1
29           1
Name: count, dtype: int64